In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
# Load data
df = pd.read_csv('linear_regression_3.csv')
X = df.drop(columns=['y'])
Y = df['y']

In [ ]:
# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=25)

In [ ]:
# 2. Normalize with index preservation
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

In [ ]:
# Ensure Y is properly indexed
Y_train = pd.Series(Y_train.values, index=X_train.index)
Y_test = pd.Series(Y_test.values, index=X_test.index)

In [ ]:
# 3. Fit initial model with aligned indices
X_train_sm = sm.add_constant(X_train_scaled)
model = sm.OLS(Y_train, X_train_sm).fit()  # Now indices match
print("Initial Model Summary:")
print(model.summary())

In [ ]:
# 4. Check VIF and remove high multi collinearity
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i)
                       for i in range(data.shape[1])]
    return vif_data


def remove_high_vif_iteratively(data, threshold=10, max_iter=20):
    """
    Removes high-VIF features one at a time, recomputing VIF after each removal.
    Preserves features that are statistically significant despite high VIF.
    """
    for _ in range(max_iter):
        vif = calculate_vif(data)
        max_vif = vif['VIF'].max()

        if max_vif <= threshold:
            break

        # Get the feature with highest VIF
        worst_feature = vif.loc[vif['VIF'].idxmax(), 'feature']

        # Check if the feature is statistically significant
        temp_model = sm.OLS(Y_train, sm.add_constant(data)).fit()
        p_value = temp_model.pvalues[worst_feature]

        # Only remove if insignificant (p > 0.05)
        if p_value > 0.05:
            print(f"Removing {worst_feature} (VIF={max_vif:.1f}, p={p_value:.3f})")
            data = data.drop(columns=[worst_feature])
        else:
            print(f"Keeping {worst_feature} despite high VIF ({max_vif:.1f}) because it's significant (p={p_value:.3f})")
            break

    return data

# Apply iterative removal
print("\nStarting VIF reduction process...")
X_train_reduced = remove_high_vif_iteratively(X_train_scaled.copy())
X_test_reduced = X_test_scaled[X_train_reduced.columns]

# Verify final VIFs
final_vif = calculate_vif(X_train_reduced)
print("\nFinal VIF Scores:")
print(final_vif)

# Check model performance
if not X_train_reduced.empty:
    X_train_sm = sm.add_constant(X_train_reduced)
    model_reduced = sm.OLS(Y_train, X_train_sm).fit()
    print(f"\nModel R² after VIF reduction: {model_reduced.rsquared:.3f} ")
else:
    print("Warning: All features removed by VIF threshold!")

In [ ]:
# 6. Re-fit model after VIF reduction
if not X_train_reduced.empty:
    X_train_sm = sm.add_constant(X_train_reduced)
    model_reduced = sm.OLS(Y_train, X_train_sm).fit()
    print("\nReduced Model Summary (after VIF):")
    print(model_reduced.summary())

    # Check for features with borderline VIF (4-10) and high p-values
    vif_reduced = calculate_vif(X_train_reduced)
    p_values = model_reduced.pvalues[1:]  # Exclude intercept

    # Identify candidate features for removal
    candidate_features = []
    for feature in X_train_reduced.columns:
        vif_val = vif_reduced[vif_reduced['feature'] == feature]['VIF'].values[0]
        p_val = p_values[feature]
        if (4 < vif_val <= 10) and (p_val > 0.05):
            candidate_features.append(feature)

    # Cross-validation to decide on removing borderline features
    current_features = X_train_reduced.columns.tolist()
    for feature in candidate_features:
        # Check if feature is still present
        if feature not in current_features:
            continue

        # Cross-validation comparison
        scores_with = cross_val_score(LinearRegression(),
                                      X_train_reduced[current_features], Y_train,
                                      cv=5, scoring='r2').mean()
        scores_without = cross_val_score(LinearRegression(),
                                         X_train_reduced[current_features].drop(columns=feature), Y_train,
                                         cv=5, scoring='r2').mean()

        if scores_without > scores_with:
            current_features.remove(feature)

    X_train_reduced = X_train_reduced[current_features]
    X_test_reduced = X_test_reduced[current_features]

In [ ]:
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i)
                       for i in range(data.shape[1])]
    return vif_data

def remove_high_vif_iteratively(data, y, threshold=10, max_iter=20):
    """
    Removes high-VIF features one at a time, recomputing VIF after each removal.
    Preserves features that are statistically significant despite high VIF.
    """
    for _ in range(max_iter):
        vif = calculate_vif(data)
        max_vif = vif['VIF'].max()

        if max_vif <= threshold:
            break

        worst_feature = vif.loc[vif['VIF'].idxmax(), 'feature']
        temp_model = sm.OLS(y, sm.add_constant(data)).fit()
        p_value = temp_model.pvalues[worst_feature]

        if p_value > 0.05:
            print(f"Removing {worst_feature} (VIF={max_vif:.1f}, p={p_value:.3f})")
            data = data.drop(columns=[worst_feature])
        else:
            print(f"Keeping {worst_feature} despite high VIF ({max_vif:.1f}) because significant (p={p_value:.3f})")
            break

    return data

def handle_borderline_vif(X, y, vif_lower=4, vif_upper=10, p_threshold=0.05):
    """
    Handles features with borderline VIF using cross-validation.
    Only removes features if they have high p-value AND removal improves CV R².
    """
    vif = calculate_vif(X)
    model = sm.OLS(y, sm.add_constant(X)).fit()
    p_values = model.pvalues[1:]  # Exclude intercept

    candidates = []
    for feature in X.columns:
        vif_val = vif[vif['feature'] == feature]['VIF'].values[0]
        p_val = p_values[feature]
        if (vif_lower < vif_val <= vif_upper) and (p_val > p_threshold):
            candidates.append(feature)

    current_features = X.columns.tolist()
    for feature in candidates:
        if feature not in current_features:
            continue

        scores_with = cross_val_score(LinearRegression(),
                                      X[current_features], y,
                                      cv=5, scoring='r2').mean()
        scores_without = cross_val_score(LinearRegression(),
                                         X[current_features].drop(columns=feature), y,
                                         cv=5, scoring='r2').mean()

        if scores_without > scores_with:
            current_features.remove(feature)
            print(f"Removing {feature} (VIF={vif_val:.1f}, p={p_val:.3f}) improved CV R²")

    return X[current_features]

# Main VIF handling pipeline
print("\n=== Starting VIF Reduction Process ===")

# 1. First remove high VIF features iteratively
print("\nStep 1: Removing high-VIF features (VIF > 10)...")
X_train_reduced = remove_high_vif_iteratively(X_train_scaled.copy(), Y_train)
X_test_reduced = X_test_scaled[X_train_reduced.columns]

# 2. Handle borderline VIF features with cross-validation
if not X_train_reduced.empty:
    print("\nStep 2: Checking borderline VIF features (4 < VIF ≤ 10)...")
    X_train_reduced = handle_borderline_vif(X_train_reduced, Y_train)
    X_test_reduced = X_test_reduced[X_train_reduced.columns]

# 3. Final verification
if not X_train_reduced.empty:
    final_vif = calculate_vif(X_train_reduced)
    print("\n=== Final Results ===")
    print("Remaining Features:", X_train_reduced.columns.tolist())
    print("\nFinal VIF Scores:")
    print(final_vif)

    final_model = sm.OLS(Y_train, sm.add_constant(X_train_reduced)).fit()
    print(f"\nFinal Model R²: {final_model.rsquared:.3f}")
    print("\nModel Summary:")
    print(final_model.summary())
else:
    print("Warning: All features removed during VIF reduction!")

In [ ]:
if not X_train_reduced.empty:
    X_train_sm = sm.add_constant(X_train_reduced)
    model_reduced = sm.OLS(Y_train, X_train_sm).fit()
    influence = OLSInfluence(model_reduced)

    # Get Cook's distance (returns array of distances and scalar threshold)
    cook_d = influence.cooks_distance[0]  # Array of distances for each observation
    cook_threshold = 4 / (len(X_train_reduced) - X_train_reduced.shape[1] - 1)  # Manual threshold calc

    # Get DFFITS
    dffits = influence.dffits[0]
    dffits_threshold = 2 * np.sqrt((X_train_reduced.shape[1] + 1)/len(X_train_reduced))

    # Identify outliers
    outlier_mask = (cook_d > cook_threshold) | (np.abs(dffits) > dffits_threshold)

    print(f"\nOutlier Statistics:")
    print(f"• Max Cook's D: {np.max(cook_d):.4f} (Threshold: {cook_threshold:.4f})")
    print(f"• Max |DFFITS|: {np.max(np.abs(dffits)):.4f} (Threshold: {dffits_threshold:.4f})")
    print(f"Found {np.sum(outlier_mask)} influential outliers ({np.mean(outlier_mask):.1%} of data)")

    # Remove outliers
    X_train_clean = X_train_reduced.loc[~outlier_mask]
    Y_train_clean = Y_train.loc[~outlier_mask]

In [ ]:
# 8. Final model fitting
if not X_train_clean.empty:
    X_train_final = sm.add_constant(X_train_clean)
    final_model = sm.OLS(Y_train_clean, X_train_final).fit()
    print("\nFinal Model Summary:")
    print(final_model.summary())

    # 9. Residual analysis
    residuals = final_model.resid
    fitted = final_model.fittedvalues

    plt.figure(figsize=(12, 8))

    # Residuals vs Fitted
    plt.subplot(221)
    plt.scatter(fitted, residuals, alpha=0.6)
    plt.axhline(0, color='r', linestyle='--')
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.title('Residuals vs Fitted')

    # Q-Q Plot
    plt.subplot(222)
    stats.probplot(residuals, dist="norm", plot=plt)
    plt.title('Normal Q-Q Plot')

    # Residual Histogram
    plt.subplot(223)
    plt.hist(residuals, bins=30, edgecolor='k')
    plt.title('Residual Distribution')

    # Scale-Location Plot
    plt.subplot(224)
    plt.scatter(fitted, np.sqrt(np.abs(residuals)), alpha=0.6)
    plt.xlabel('Fitted Values')
    plt.ylabel('√|Standardized Residuals|')
    plt.title('Scale-Location Plot')

    plt.tight_layout()
    plt.show()

    # 10. Test set evaluation
    X_test_final = sm.add_constant(X_test_reduced[X_train_clean.columns], has_constant='add')
    test_predictions = final_model.predict(X_test_final)
    test_residuals = Y_test - test_predictions

    # Plot test residual distribution with IQR bounds
    Q1 = np.percentile(test_residuals, 25)
    Q3 = np.percentile(test_residuals, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    plt.figure(figsize=(14, 6))
    plt.hist(test_residuals, bins=30, edgecolor='k', alpha=0.7)
    plt.axvline(lower_bound, color='r', linestyle='--', label='Lower Bound')
    plt.axvline(upper_bound, color='r', linestyle='--', label='Upper Bound')
    plt.title('Test Set Residual Distribution with IQR Bounds')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

    # Calculate performance metrics
    test_mask = (test_residuals >= lower_bound) & (test_residuals <= upper_bound)

    # Full test set metrics
    test_r2 = r2_score(Y_test, test_predictions)
    test_rmse = np.sqrt(mean_squared_error(Y_test, test_predictions))

    # Clean test set metrics (non-outliers only)
    test_r2_clean = r2_score(Y_test[test_mask], test_predictions[test_mask])
    test_rmse_clean = np.sqrt(mean_squared_error(Y_test[test_mask], test_predictions[test_mask]))
    outlier_percentage = (1 - test_mask.mean()) * 100

    print("\nModel Performance:")
    print(f"Train Adjusted R²: {final_model.rsquared_adj:.3f}")
    print(f"\nTest Set Performance (Full):")
    print(f"R²: {test_r2:.3f} | RMSE: {test_rmse:.3f} | n={len(Y_test)}")
    print(f"\nTest Set Performance (Non-outliers only):")
    print(f"R²: {test_r2_clean:.3f} | RMSE: {test_rmse_clean:.3f} | n={test_mask.sum()} ({100-outlier_percentage:.1f}% retained)")
    print(f"\nOutlier Statistics:")
    print(f"• Residual IQR: [{Q1:.3f}, {Q3:.3f}]")
    print(f"• Outlier bounds: [{lower_bound:.3f}, {upper_bound:.3f}]")
    print(f"• Outlier percentage: {outlier_percentage:.1f}%")

else:
    print("No features remaining after feature selection.")